In [1]:
from datetime import datetime


In [18]:
class WidgetException(Exception):
    message = 'Generic Widget Exception'
    
    def __init__(self,*args,custom_message = None):
        super().__init__(args)
        if args:
            self.message = args[0]
        self.custom_message = custom_message if custom_message is not None else self.message
    
    def log_exception(self):
        exception={
            "type":type(self).__name__,
            "message":self.message,
            "args": self.args[1:]
        }
        print(f'EXCEPTION:{datetime.utcnow().isoformat()}: {exception}')

In [19]:
ex1 = WidgetException('some custom message',10,100)
ex2 = WidgetException(custom_message = 'A custom user message')

In [20]:
ex1.log_exception()

EXCEPTION:2020-03-28T10:36:41.273978: {'type': 'WidgetException', 'message': 'some custom message', 'args': ()}


In [21]:
ex2.log_exception()

EXCEPTION:2020-03-28T10:36:41.907259: {'type': 'WidgetException', 'message': 'Generic Widget Exception', 'args': ()}


In [26]:
class SupplierException(WidgetException):
    message = 'Supplier excetpion'
    
class NotManufacturedException(SupplierException):
    message = 'Widget is no longer manufactured by supplier'

class ProductionDelayedException(SupplierException):
    message = 'Widget production has been delayed by manufacturer'
    
class ShippingDelayedException(SupplierException):
    message = 'Widget shipping has been delayed by supplier'
    
class CheckoutException(WidgetException):
    message = 'Checkout exception'
    
class InventoryException(CheckoutException):
    message = 'Checkout inventory exception'

class OutOfStockExcetion(InventoryException):
    message = 'Inventory out of stock'
    
class PricingException(CheckoutException):
    message = 'Checkout pricing exception'
    
class InvalidCouponCodeException(PricingException):
    messge = 'Invalid checkout coupon code'
    
class CannotStackCouponException(PricingException):
    message = 'Cannot stack checkout coupon code'

In [27]:
try:
    raise CannotStackCouponException()
except CannotStackCouponException as ex:
    ex.log_exception()
    raise

EXCEPTION:2020-03-28T10:46:56.678672: {'type': 'CannotStackCouponException', 'message': 'Cannot stack checkout coupon code', 'args': ()}


CannotStackCouponException: ()

In [35]:
from http import HTTPStatus
import json

In [36]:
class WidgetException(Exception):
    message = 'Generic Widget Exception'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    def __init__(self,*args,custom_message = None):
        super().__init__(args)
        if args:
            self.message = args[0]
        self.custom_message = custom_message if custom_message is not None else self.message
    
    def log_exception(self):
        exception={
            "type":type(self).__name__,
            "message":self.message,
            "args": self.args[1:]
        }
        print(f'EXCEPTION:{datetime.utcnow().isoformat()}: {exception}')
    
    def to_json(self):
        response = {
            'code' : self.http_status.value,
            'message':'{}:{}'.format(self.http_status.phrase,self.custom_message),
            'category':type(self).__name__,
            'time_utc':datetime.utcnow().isoformat()
        }
        return json.dumps(response)

In [37]:
e = WidgetException('same custom message for log and user')

In [38]:
e.log_exception()

EXCEPTION:2020-03-28T10:52:17.799460: {'type': 'WidgetException', 'message': 'same custom message for log and user', 'args': ()}


In [39]:
json.loads(e.to_json())

{'code': 500,
 'message': 'Internal Server Error:same custom message for log and user',
 'category': 'WidgetException',
 'time_utc': '2020-03-28T10:52:18.263759'}

In [40]:
e = WidgetException('custom internal message',custom_message = 'custom user message')
e.log_exception()

EXCEPTION:2020-03-28T10:53:07.104740: {'type': 'WidgetException', 'message': 'custom internal message', 'args': ()}


In [41]:
e.to_json()

'{"code": 500, "message": "Internal Server Error:custom user message", "category": "WidgetException", "time_utc": "2020-03-28T10:53:14.529622"}'

In [42]:
try:
    raise WidgetException('custom error message')
except WidgetException as ex:
    print(repr(ex.__traceback__))

In [43]:
import traceback

In [46]:
try:
    raise ValueError
except ValueError:
    try:
        raise WidgetException('custom message')
    except WidgetException as ex:
        tb = traceback.TracebackException.from_exception(ex)
        print(list(tb.format()))

['Traceback (most recent call last):\n', '  File "<ipython-input-46-4f5d3f96385f>", line 2, in <module>\n    raise ValueError\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "<ipython-input-46-4f5d3f96385f>", line 5, in <module>\n    raise WidgetException(\'custom message\')\n', "WidgetException: ('custom message',)\n"]


In [47]:
try:
    raise ValueError
except ValueError:
    try:
        raise WidgetException('custom message')
    except WidgetException as ex:
        tb = traceback.TracebackException.from_exception(ex)
        tb = ''.join(tb.format())
        print(tb)

Traceback (most recent call last):
  File "<ipython-input-47-d1860de49ad0>", line 2, in <module>
    raise ValueError
ValueError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-47-d1860de49ad0>", line 5, in <module>
    raise WidgetException('custom message')
WidgetException: ('custom message',)



In [48]:
try:
    raise ValueError
except ValueError:
    raise WidgetException('custom message')

WidgetException: ('custom message',)

In [57]:
class WidgetException(Exception):
    message = 'Generic Widget Exception'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    def __init__(self,*args,custom_message = None):
        super().__init__(args)
        if args:
            self.message = args[0]
            self.custom_message = custom_message if custom_message is not None else self.message
    
    @property
    def traceback(self):
        return traceback.TracebackException.from_exception(self).format()
    
    def log_exception(self):
        exception={
            "type":type(self).__name__,
            "message":self.message,
            "args": self.args[1:],
            "traceback":list(self.traceback)
        }
        print(f'EXCEPTION:{datetime.utcnow().isoformat()}: {exception}')
    
    def to_json(self):
        response = {
            'code' : self.http_status.value,
            'message':'{}:{}'.format(self.http_status.phrase,self.custom_message),
            'category':type(self).__name__,
            'time_utc':datetime.utcnow().isoformat()
        }
        return json.dumps(response)

In [56]:
try:
    raise WidgetException()
except WidgetException as ex:
    ex.log_exception()
    print('-----------')
    print(ex.to_json())

EXCEPTION:2020-03-28T11:07:05.258664: {'type': 'WidgetException', 'message': 'Generic Widget Exception', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "<ipython-input-56-9bb26bb9376b>", line 2, in <module>\n    raise WidgetException()\n', 'WidgetException: ()\n']}
-----------


AttributeError: 'WidgetException' object has no attribute 'custom_message'

In [60]:
try:
    a = 1/0
except ZeroDivisionError:
    try:
        raise WidgetException()
    except WidgetException as ex:
            print(''.join(ex.traceback))

Traceback (most recent call last):
  File "<ipython-input-60-7f2699ae1512>", line 2, in <module>
    a = 1/0
ZeroDivisionError: division by zero

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-60-7f2699ae1512>", line 5, in <module>
    raise WidgetException()
WidgetException: ()



In [61]:
class SupplierException(WidgetException):
    message = 'Supplier excetpion'
    
class NotManufacturedException(SupplierException):
    message = 'Widget is no longer manufactured by supplier'

class ProductionDelayedException(SupplierException):
    message = 'Widget production has been delayed by manufacturer'
    
class ShippingDelayedException(SupplierException):
    message = 'Widget shipping has been delayed by supplier'
    
class CheckoutException(WidgetException):
    message = 'Checkout exception'
    
class InventoryException(CheckoutException):
    message = 'Checkout inventory exception'

class OutOfStockExcetion(InventoryException):
    message = 'Inventory out of stock'
    
class PricingException(CheckoutException):
    message = 'Checkout pricing exception'
    
class InvalidCouponCodeException(PricingException):
    messge = 'Invalid checkout coupon code'
    
class CannotStackCouponException(PricingException):
    message = 'Cannot stack checkout coupon code'

In [64]:
e = InvalidCouponCodeException(
    'User tried to pull a fast one on us',custom_message = 'Sorry,this coupon is not valid'
)

In [65]:
e.log_exception()

EXCEPTION:2020-03-28T11:11:44.343019: {'type': 'InvalidCouponCodeException', 'message': 'User tried to pull a fast one on us', 'args': (), 'traceback': ["InvalidCouponCodeException: ('User tried to pull a fast one on us',)\n"]}


In [66]:
e.to_json()

'{"code": 500, "message": "Internal Server Error:Sorry,this coupon is not valid", "category": "InvalidCouponCodeException", "time_utc": "2020-03-28T11:11:50.118210"}'

In [68]:
try:
    raise ValueError
except ValueError:
    try:
        raise InvalidCouponCodeException(
    'User tried to pull a fast one on us',custom_message = 'Sorry,this coupon is not valid'
)
    except InvalidCouponCodeException as ex:
        ex.log_exception()
        print('------')
        print(ex.to_json)
        print('------')
        print(''.join(ex.traceback))

EXCEPTION:2020-03-28T11:13:21.321867: {'type': 'InvalidCouponCodeException', 'message': 'User tried to pull a fast one on us', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "<ipython-input-68-23d9f2239167>", line 2, in <module>\n    raise ValueError\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "<ipython-input-68-23d9f2239167>", line 6, in <module>\n    \'User tried to pull a fast one on us\',custom_message = \'Sorry,this coupon is not valid\'\n', "InvalidCouponCodeException: ('User tried to pull a fast one on us',)\n"]}
------
<bound method WidgetException.to_json of InvalidCouponCodeException(('User tried to pull a fast one on us',),)>
------
Traceback (most recent call last):
  File "<ipython-input-68-23d9f2239167>", line 2, in <module>
    raise ValueError
ValueError

During handling of the above exception, another exception occurred:

Traceback (most recen